In [1]:
import argparse
import pathlib
import random
import warnings

import numpy as np
import pandas as pd
from copairs import map
from copairs.matching import assign_reference_index

# suppress warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [2]:
well_to_dose_path = pathlib.Path(
    "../../data/CP_scDINO_features/combined_CP_scDINO_norm_fs.parquet"
).resolve(strict=True)
well_to_dose_df = pd.read_parquet(
    well_to_dose_path, columns=["Metadata_Well", "Metadata_dose"]
)
# drop duplicates
well_to_dose_df = well_to_dose_df.drop_duplicates(
    subset=["Metadata_Well", "Metadata_dose"]
)
# make dict that maps well to dose
well_to_dose = well_to_dose_df.set_index("Metadata_Well")["Metadata_dose"].to_dict()

In [3]:
CP_scDINO_profile_file_path = pathlib.Path(
    "../../data/CP_scDINO_features/combined_CP_scDINO_norm_fs_aggregated.parquet"
).resolve(strict=True)
df = pd.read_parquet(CP_scDINO_profile_file_path)
df.head()

,Metadata_Well,Metadata_Time,Metadata_dose,Cells_AreaShape_BoundingBoxArea_CP,Cells_AreaShape_Compactness_CP,Cells_AreaShape_Extent_CP,Cells_AreaShape_FormFactor_CP,Cells_AreaShape_MinorAxisLength_CP,Cells_AreaShape_Solidity_CP,Cells_AreaShape_Zernike_1_1_CP,...,channel_DNA_cls_feature_91_scDINO,channel_DNA_cls_feature_92_scDINO,channel_DNA_cls_feature_93_scDINO,channel_DNA_cls_feature_94_scDINO,channel_DNA_cls_feature_95_scDINO,channel_DNA_cls_feature_96_scDINO,channel_DNA_cls_feature_97_scDINO,channel_DNA_cls_feature_98_scDINO,channel_DNA_cls_feature_99_scDINO,channel_DNA_cls_feature_9_scDINO
0,C-02,0.0,0.0,-0.291760,-0.312209,0.311346,0.138225,-0.253509,0.408095,-0.049747,...,0.208938,-0.073630,0.008897,-0.055376,0.131249,-0.053869,0.121860,0.006765,-0.151495,0.121353
1,C-02,1.0,0.0,-0.028198,-0.394414,0.313888,0.260440,0.312064,0.460261,-0.253127,...,-0.080875,0.047516,-0.007985,0.057872,-0.307675,0.226858,-0.097667,0.154435,0.009346,-0.100267
2,C-02,10.0,0.0,0.182481,-0.209134,0.137440,-0.006370,0.309906,0.357322,-0.434222,...,-0.104633,0.049896,0.155820,0.073132,-0.514341,0.125680,0.012130,0.206564,-0.001114,-0.069553
3,C-02,11.0,0.0,0.141672,-0.285466,0.255858,0.099816,0.399589,0.385191,-0.438585,...,-0.087787,-0.101442,0.275884,0.209213,-0.459635,0.113541,-0.091702,0.251352,0.081938,-0.135812
4,C-02,12.0,0.0,0.152554,-0.308668,0.294745,0.133101,0.450444,0.414189,-0.436021,...,0.009973,-0.036503,0.473306,0.311591,-0.641266,0.173728,0.024222,0.272495,0.057495,-0.097702


In [4]:
metadata_columns = [x for x in df.columns if "Metadata" in x]
scDINO_columns = [x for x in df.columns if "scDINO" in x]
CP_columns = df.drop(columns=metadata_columns + scDINO_columns).columns
CP_scDINO_columns = df.drop(metadata_columns, axis=1).columns

feature_set_dict = {
    "scDINO": scDINO_columns,
    "CP": CP_columns,
    "CP_scDINO": CP_scDINO_columns,
}

In [5]:
df["Metadata_Time"] = df["Metadata_Time"].astype(float)
timepoints = df["Metadata_Time"].unique().tolist()
timepoints.sort()
shuffle_options = [False, True]

In [6]:
# This loops through the CP, scDINO, and CP_scDINO feature sets and calculates the mean average precision for each timepoint


for feature_set_name, feature_set in feature_set_dict.items():
    feature_subset_df = pd.concat([df[feature_set], df[metadata_columns]], axis=1)

    list_of_maps = []
    # calculate the mean average precision for each timepoint
    # and each shuffle option
    for timepoint in timepoints:
        time_subset_df = feature_subset_df[
            feature_subset_df["Metadata_Time"] == timepoint
        ]

        for shuffle in shuffle_options:
            if shuffle == True:
                random.seed(0)
                # permutate the data
                for col in time_subset_df.columns:
                    if col != "Metadata_dose":
                        time_subset_df.loc[:, col] = np.random.permutation(
                            time_subset_df[col]
                        )

            reference_col = "Metadata_reference_index"
            df_activity = assign_reference_index(
                time_subset_df,
                "Metadata_dose == '0.0'",
                reference_col=reference_col,
                default_value=-1,
            )
            pos_sameby = ["Metadata_dose", reference_col]
            pos_diffby = []
            neg_sameby = []
            neg_diffby = ["Metadata_dose", reference_col]
            metadata = df_activity.filter(regex="Metadata")
            profiles = df_activity.filter(regex="^(?!Metadata)")
            # drop the nans from the profiles
            profiles = profiles.dropna(axis=0)
            metadata = metadata.loc[profiles.index]
            profiles = profiles.values
            # suppress the warning
            activity_ap = map.average_precision(
                metadata, profiles, pos_sameby, pos_diffby, neg_sameby, neg_diffby
            )

            activity_ap = activity_ap.query("Metadata_dose != '0.0'")
            activity_map = map.mean_average_precision(
                activity_ap, pos_sameby, null_size=1000000, threshold=0.05, seed=0
            )
            activity_map.reset_index(inplace=True)
            activity_map.insert(
                0,
                "Metadata_Time",
                timepoint,
            )
            if shuffle:
                activity_map.insert(
                    1,
                    "Shuffle",
                    "True",
                )
            else:
                activity_map.insert(
                    1,
                    "Shuffle",
                    "False",
                )
            list_of_maps.append(activity_map)

    final_mAP_df = pd.concat(list_of_maps, ignore_index=True)
    # save the final dataframe to a parquet file
    final_mAP_path = pathlib.Path(
        f"../data/mAP/mAP_scores_{feature_set_name}.parquet"
    ).resolve()
    final_mAP_path.parent.mkdir(parents=True, exist_ok=True)
    final_mAP_df.to_parquet(
        final_mAP_path,
        index=False,
    )

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]